In [1]:
import pandas as pd
import requests

In [2]:
data = pd.read_csv("NewLinks2017-12-03 20:28:18.862583.csv")

In [3]:
roster = data[data.key == 'roster'].drop_duplicates()

In [4]:
roster.sort_values(['team'], inplace = True)

In [6]:
roster_list = roster.to_dict(orient='record')

In [7]:
r = requests.get(roster_list[0]['link'])

In [8]:
tables = pd.read_html(r.content)

In [9]:
len(tables)

1

In [20]:
tables[0].columns = tables[0].columns.droplevel(0)

In [21]:
tables[0]

,Jersey,Player,Pos,Yr,GP,GS
0,59,"Adenji, Moshood",OL,Sr,12,12
1,35,"Alabi, Justin",LB,Fr,6,0
2,12,"Awini, Jaleel",QB,So,0,0
3,28,"Baker, Kalon",DB,So,5,0
4,18,"Barnes, Reggie",LB,So,12,11
5,11,"Barnes, Myles",WR,So,10,2
6,19,"Baska, David",K,Sr,12,0
7,23,"Batts, Steffon",DB,Sr,12,12
8,98,"Beschel, Dillon",DL,Jr,7,0
9,7,"Brown, Garrett",WR,So,12,2
